In [169]:
from astroquery.simbad import Simbad
import pandas as pd

In [170]:
Simbad.ROW_LIMIT = 15 # now any query fetches at most 15 rows for testing purposes
Simbad.reset_votable_fields()
Simbad.add_votable_fields("distance", "flux(K)", "flux(G)", "flux(R)", "flux(B)")

In [171]:
test_table = Simbad.query_criteria(maintype="RR?")

In [172]:
df = test_table.to_pandas()
df

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,FLUX_K,FLUX_G,FLUX_R,FLUX_B,SCRIPT_NUMBER_ID
0,V* V453 Per,03 06 07.2568,+36 40 05.158,14,14,0.0658,0.0500,90,A,O,...,,NaN,NaN,,,15.744,16.510054,NaN,NaN,0
1,ATO J355.5144+52.1462,23 42 03.4788,+52 08 46.448,14,14,0.0222,0.0218,90,A,O,...,,NaN,NaN,,,14.294,15.701858,NaN,NaN,0
2,V* V1392 Cyg,20 32 36.4509,+40 07 28.009,14,14,0.5130,0.5415,90,A,O,...,,NaN,NaN,,,11.835,16.273645,NaN,NaN,0
3,ATO J352.4259+55.0630,23 29 42.2230,+55 03 47.099,14,14,0.0167,0.0169,90,A,O,...,,NaN,NaN,,,13.720,15.228209,15.532000,15.806000,0
4,TYC 3592-835-1,21 00 07.8708,+48 41 29.556,14,14,0.0144,0.0145,90,A,O,...,pc,-5.0852,5.0852,paral,2020yCat.1350....0G,7.124,10.257861,10.110000,11.390000,0
5,V* NU Lac,22 31 03.3248,+55 15 36.983,14,14,0.0281,0.0273,90,A,O,...,pc,-252.7087,252.7087,paral,2020yCat.1350....0G,13.965,15.962180,16.290001,16.840000,0
6,V* V845 Cas,23 26 15.0452,+57 23 55.380,14,14,0.0137,0.0128,90,A,O,...,pc,-137.0493,137.0493,paral,2020yCat.1350....0G,11.945,14.323001,14.329000,15.659000,0
7,V* V508 Per,02 33 01.1312,+58 25 37.278,14,14,0.0240,0.0248,90,A,O,...,,NaN,NaN,,,13.675,15.763356,15.870000,16.420000,0
8,TYC 4253-145-1,21 41 30.7796,+62 48 06.779,14,14,0.0496,0.0608,90,A,O,...,pc,-36.4727,36.4727,paral,2020yCat.1350....0G,9.459,11.091930,NaN,11.500000,0
9,V* V416 Per,04 08 08.9558,+45 24 10.680,14,14,0.0208,0.0131,90,A,O,...,pc,-45.4766,45.4766,paral,2020yCat.1350....0G,12.789,14.724621,NaN,NaN,0


# Scrapping the Christine Clement database of variable stars

This database has a comprehensive list of a lot of pulsating stars in different clusters with their type and magnitude listed. We are going to extract all the stars in the different clusters, and name them in the following manner:

`Cluster_IDnumber`

We will then try to find their ID in the SIMBAD database. Hopefully there is some database out there that has the names of the different stars in a cluster, but if it does not exist we will attempt to use the coordinates of the star to find it in SIMBAD.

In [173]:
from selenium import webdriver

In [186]:
driver = 'C:/Users/cuco2/OneDrive/Escritorio/Physics/Year 4/Final year project/MphysProject/Drivers/chromedriver.exe'
options = webdriver.ChromeOptions()
# options.add_argument('headless')

driver = webdriver.Chrome(executable_path = driver, options = options)

In [187]:
url = "http://www.astro.utoronto.ca/~cclement/cat/listngc.html"
driver.get(url)

In [188]:
cluster_list_imported = driver.find_elements_by_xpath("//a[@href]")
cluster_list_cleaned = [elem.get_attribute("href") for elem in cluster_list_imported]

In [189]:
# We're removing the list with the last updates
cluster_list_cleaned = cluster_list_cleaned[1:]

In [190]:
# Create a function to extract the data from each of the cluster subpages

import re
from io import StringIO

def extract_cluster_data(text):
    # Create a dictionary to get rid of all the equal signs (=) as line breaks
    dict_list = re.split(r"=", text)
    dict_list = list(dict.fromkeys(dict_list))

    # The actual stars are the 8th element of the list
    # Save as StringIO so we can put it in a df later
    data = StringIO(dict_list[7])

    # This contains the information in each column of the table, the 6th element in the list
    header = dict_list[6]
    header = header.split(" ")
    header = list(dict.fromkeys(header))
    header.remove('')

    df = pd.read_csv(data, sep = "\s+", names = header)

    # Obtain the clusterName, the first element in the list
    clusterName = list(re.findall(">.*\n*", dict_list[3])[0])
    clusterName.remove('>')
    clusterName.remove('\n')
    clusterName = ''.join(clusterName)

    # Rename the ID column so each one is called clusterName_number
    for i in range(len(df["\nID"])):
        df.loc[:,("\nID", i)] = f"{clusterName}_{i+1}"

    return df

In [194]:
# create an empty df

all_variable_stars = pd.DataFrame()

for url in cluster_list_cleaned[3:]:
    driver.get(url)
    cluster_data = driver.page_source
    variable_stars = extract_cluster_data(cluster_data)
    break

ParserError: Error tokenizing data. C error: Expected 12 fields in line 3, saw 13


In [195]:
cluster_list_cleaned[3]

'http://www.astro.utoronto.ca/~cclement/cat/C0310m554'

In [180]:
variable_stars

,\nID,Position,Period,&lt;mag&gt;,ampl,C,Type,Notes/\n,RA/X,Dec/Y,...,23,24,25,26,27,28,29,30,31,32
0,NGC 2808 C0911-646_33,09:12:20.21,-64:52:20.5,R0,119.0,13.53,1.4,V,SR,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
1,NGC 2808 C0911-646_33,09:11:55.66,-64:51:10.4,R0,----,--,--,CST,Note,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
2,NGC 2808 C0911-646_33,09:12:08.38,-64:52:47.2,R0,----,--,--,CST,NaN,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
3,NGC 2808 C0911-646_33,09:11:33.75,-64:50:44.0,R0,----,--,--,CST,Note,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
4,NGC 2808 C0911-646_33,09:12:09.58,-64:52:52.6,R0,----,--,--,CST,NaN,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
5,NGC 2808 C0911-646_33,09:12:30.08,-64:56:37.9,R0,0.5390,16.27,0.80,V,RR0,K11,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
6,NGC 2808 C0911-646_33,09:12:13.13,-64:50:46.4,R0,----,--,--,CST,NaN,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
7,NGC 2808 C0911-646_33,09:12:06.56,-64:53:49.3,R0,----,--,--,CST,NaN,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
8,NGC 2808 C0911-646_33,09:11:53.06,-64:54:24.3,R0,----,--,--,CST,NaN,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33
9,NGC 2808 C0911-646_33,09:11:56.85,-64:53:23.1,R0,1.7653,15.56,1.00,B,CW,NaN,...,NGC 2808 C0911-646_24,NGC 2808 C0911-646_25,NGC 2808 C0911-646_26,NGC 2808 C0911-646_27,NGC 2808 C0911-646_28,NGC 2808 C0911-646_29,NGC 2808 C0911-646_30,NGC 2808 C0911-646_31,NGC 2808 C0911-646_32,NGC 2808 C0911-646_33


In [181]:
for url in cluster_list_cleaned:
    print(driver.get(url))


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [182]:


mylist = re.split(r"=", h)
mylist = list(dict.fromkeys(mylist))
data = StringIO(mylist[7])

header = mylist[6]
header = header.split(" ")
header = list(dict.fromkeys(header))
header.remove('')

df = pd.read_csv(data, sep = "\s+", names = header)
df

,\nID,Position,Period,&lt;mag&gt;,ampl,C,Type,Notes/\n,RA/X,Dec/Y,Units,Remarks\n
0,1,00:52:41.13,-26:33:27.2,R0,103.0000,12.40,0.22,V,SR,Note,NaN,NaN
1,2,00:52:46.64,-26:34:07.4,R0,0.6777,15.18,1.12,V,RR0,NaN,NaN,NaN
2,3,00:52:40.21,-26:32:28.8,R0,0.4302,15.14,0.45,V,RR1,NaN,NaN,NaN
3,4,00:52:42.80,-26:34:45.4,R0,0.0791,17.24,0.30,V,SXPHE,NaN,NaN,NaN
4,5,00:52:44.98,-26:33:52.1,R0,0.0511,17.54,0.46,V,SXPHE,NaN,NaN,NaN
5,6,00:52:42.40,-26:34:54.6,R0,0.0672,17.28,0.41,V,SXPHE,NaN,NaN,NaN
6,7,00:52:41.39,-26:33:59.7,R0,0.0400,17.92,0.06,V,SXPHE,NaN,NaN,NaN
7,8,00:52:44.27,-26:33:59.6,R0,0.0465,17.78,0.06,V,SXPHE,NaN,NaN,NaN
8,9,00:52:42.88,-26:34:09.6,R0,0.0394,17.52,0.05,V,SXPHE,NaN,NaN,NaN
9,10,00:52:47.86,-26:33:01.9,R0,0.4387,19.09,0.38,V,EC,max;Note,NaN,NaN


In [183]:
clusterName = list(re.findall(">.*\n*", mylist[3])[0])
clusterName.remove('>')
clusterName.remove('\n')
clusterName = ''.join(clusterName)

In [184]:
for i in range(len(df["\nID"])):
    df["\nID"].loc[i] = f"{clusterName}_{i+1}"

C:\Users\cuco2\AppData\Local\Temp\ipykernel_26100\2954064726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["\nID"].loc[i] = f"{clusterName}_{i+1}"
